# 4.7 Exercise Part 1

Table of Contents

01. Importing

02. Wrangling

03. Data Quality and Consistency Checks

04. Merging Data

05. Export Data

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\31622\OneDrive\Documents\Career Foundry\Instacart Basket Analysis'

Importing Dataframes

In [3]:
# Import new customer data set
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [4]:
#Importing orders_products_aggregated.pkl dataframe
ords_prods_aggreg = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_aggregated.pkl'))

Question 4: Data Wrangling on New Customer Data set

In [5]:
# Check headings and first 10 rows of customer data set 
customers.head(10)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


In [6]:
#check customer data set shape
customers.shape

(206209, 10)

In [7]:
# rename columns in customer data set
customers.rename(columns = {'Surnam':'Last_name', 'n_dependants':'Num_of_dependants', 'fam_status':'Marital_status'}, inplace = True)

In [8]:
#checking renamed columns in customer data set
customers.head()

,user_id,First Name,Last_name,Gender,STATE,Age,date_joined,Num_of_dependants,Marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


Question 5: Complete data quality and consistency checks

In [9]:
# Check descriptive statistics of data set
customers.describe() # There are no strange or incorrect values in our descriptive statistics

,user_id,Age,Num_of_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [10]:
# Check information on customer data types
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            206209 non-null  int64 
 1   First Name         194950 non-null  object
 2   Last_name          206209 non-null  object
 3   Gender             206209 non-null  object
 4   STATE              206209 non-null  object
 5   Age                206209 non-null  int64 
 6   date_joined        206209 non-null  object
 7   Num_of_dependants  206209 non-null  int64 
 8   Marital_status     206209 non-null  object
 9   income             206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


Observation: From this output, First name has an entry of 194950, which means it is missing values of 11,259 (206209 - 194950).

In [11]:
#Checking and confirming First Name column for missing values as it is the only one with fewer non-null objects
customers['First Name'].value_counts(dropna = False) # We have confirmation that there is a NaN = 11259.

NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: First Name, Length: 208, dtype: int64

Check for mixed data types

In [12]:
#Check for mixed-types in customers.csv
for col in customers.columns.tolist():
    weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (customers[weird]) > 0:
        print(col)

First Name


In [13]:
#Change the datatype of First Name to a String
customers['First Name'] = customers['First Name'].astype('str')

In [14]:
# check if conversion is sucessful
for col in customers.columns.tolist():
  weird = (customers[[col]].applymap(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

Check missing values

In [15]:
# confirm no missing values
customers.isnull().sum() # There are no missin values, now that we changed data type to string of First_name.

user_id              0
First Name           0
Last_name            0
Gender               0
STATE                0
Age                  0
date_joined          0
Num_of_dependants    0
Marital_status       0
income               0
dtype: int64

Check duplicate values

In [16]:
#Find any duplicate entries in customer.csv and assign them to df_dups
df_dups = customers[customers.duplicated()]

In [17]:
df_dups # confirmed no duplicates in customer data set

,user_id,First Name,Last_name,Gender,STATE,Age,date_joined,Num_of_dependants,Marital_status,income


Question 6: Combining wrangled customer data with the rest of the prepared Instacart data

In [18]:
ords_prods_aggreg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434212 entries, 0 to 32434211
Data columns (total 24 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   number_of_orders        int64   
 3   orders_day_of_week      int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   product_id              int64   
 7   add_to_cart_order       int64   
 8   reordered               int64   
 9   product_name            object  
 10  aisle_id                int64   
 11  department_id           int64   
 12  prices                  float64 
 13  _merge                  category
 14  price_range_loc         object  
 15  busiest_day             object  
 16  busiest_days            object  
 17  busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  mean_price              float64 
 21  Spendi

In [19]:
#Checking the ords_prods_aggreg df size
ords_prods_aggreg.shape

(32434212, 24)

In [20]:
#Combining the two df using the user_id column
combined = ords_prods_aggreg.merge(customers, on = 'user_id')

In [21]:
combined.head()

,order_id,user_id,number_of_orders,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,First Name,Last_name,Gender,STATE,Age,date_joined,Num_of_dependants,Marital_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [22]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434212 entries, 0 to 32434211
Data columns (total 33 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   order_id                int64   
 1   user_id                 int64   
 2   number_of_orders        int64   
 3   orders_day_of_week      int64   
 4   order_hour_of_day       int64   
 5   days_since_prior_order  float64 
 6   product_id              int64   
 7   add_to_cart_order       int64   
 8   reordered               int64   
 9   product_name            object  
 10  aisle_id                int64   
 11  department_id           int64   
 12  prices                  float64 
 13  _merge                  category
 14  price_range_loc         object  
 15  busiest_day             object  
 16  busiest_days            object  
 17  busiest_period_of_day   object  
 18  max_order               int64   
 19  loyalty_flag            object  
 20  mean_price              float64 
 21  Spendi

In [23]:
combined.shape

(32434212, 33)

Question 8: Export this new dataframe as a pickle file

In [24]:
#Exporting combined customer data set with ords_prods_aggreg to pickle file
combined.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'customers_ords_prods.pkl'))